In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
years = list(range(2001,2023))
root_url = "https://www.basketball-reference.com/awards/awards_{}.html"
stats_per_game_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

#### MVPs each season

In [21]:
for year in years:
    url = root_url.format(year)
    data = requests.get(url)
        
    with open("mvps/mvp_season_{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)
        

In [21]:
dfs = []
for year in years:
    with open ("mvps/mvp_season_{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr',class_="over_header").decompose()
    mvp_table = soup.find_all(id= "mvp")
    df = pd.read_html(str(mvp_table))[0]
    df["Year"] = year
    dfs.append(df)

In [22]:
mvps = pd.concat(dfs)

In [25]:
mvps.tail()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
7,8,Stephen Curry,33,GSW,0.0,4.0,1000,0.004,64,34.5,...,5.2,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173,2022
8,9,Chris Paul,36,PHO,0.0,2.0,1000,0.002,65,32.9,...,4.4,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210,2022
9,10T,DeMar DeRozan,32,CHI,0.0,1.0,1000,0.001,76,36.1,...,5.2,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154,2022
10,10T,Kevin Durant,33,BRK,0.0,1.0,1000,0.001,55,37.2,...,7.4,6.4,0.9,0.9,0.518,0.383,0.910,8.4,0.198,2022
11,10T,LeBron James,37,LAL,0.0,1.0,1000,0.001,56,37.2,...,8.2,6.2,1.3,1.1,0.524,0.359,0.756,7.5,0.172,2022


In [26]:
mvps.to_csv("mvps.csv")

#### All player stats per game each season

In [12]:
browser = webdriver.Chrome(executable_path=r"C:\User\ledat\chromedriver.exe")

C:\Users\ledat\AppData\Local\Temp\ipykernel_15028\1519481993.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=r"C:\User\ledat\chromedriver.exe")


In [14]:
import time
for year in years:
    url = stats_per_game_url.format(year)

    browser.get(url)
    browser.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)
    
    html = browser.page_source
    
    with open("all_player_stats/all_player_stats{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(html)

In [34]:
dfs = []
for year in years:
    with open ("all_player_stats/all_player_stats{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    for remove in soup.findAll('tr',class_="thead"):
        remove.decompose()
    stats_table = soup.find(id= "per_game_stats")
    df = pd.read_html(str(stats_table))[0]
    df["Year"] = year
    dfs.append(df)


In [37]:
stats = pd.concat(dfs)
stats.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
807,601,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,5.5,...,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2022
808,602,Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,...,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2022
809,603,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,...,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2022
810,604,Cody Zeller,C,29,POR,27,0,13.1,1.9,3.3,...,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,2022
811,605,Ivica Zubac,C,24,LAC,76,76,24.4,4.1,6.5,...,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,2022


In [38]:
stats.to_csv("stats.csv")

#### Team stats each season

In [5]:
for year in years:
    data = requests.get(team_stats_url.format(year))
    
    with open("team_stats/team_stats_{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)

In [50]:
dfs = []
for year in years:
    with open("team_stats/team_stats_{}.html".format(year), "r+", encoding='utf-8') as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        for _ in soup.findAll('tr',class_="thead"):
            _.decompose()
        sides = ("Eastern Conference", "Western Conference")
        for side in sides:
            team_stats_table = soup.find_all(id= "divs_standings_{}".format(side[0]))
            df = pd.read_html(str(team_stats_table))[0]
            df["Year"] = year
            df.rename(columns={side: "Team name"}, inplace=True)
            dfs.append(df)

In [56]:
team_stats = pd.concat(dfs)
team_stats.tail()

,Team name,W,L,W/L%,GB,PS/G,PA/G,SRS,Year
10,Memphis Grizzlies*,56,26,0.683,—,115.6,109.9,5.37,2022
11,Dallas Mavericks*,52,30,0.634,4.0,108.0,104.7,3.12,2022
12,New Orleans Pelicans*,36,46,0.439,20.0,109.3,110.3,-0.84,2022
13,San Antonio Spurs,34,48,0.415,22.0,113.2,113.0,0.02,2022
14,Houston Rockets,20,62,0.244,36.0,109.7,118.2,-8.26,2022


In [57]:
team_stats.to_csv("team_stats.csv")